In [5]:
import numpy as np
from typing import Tuple, Callable, Protocol
from enum import Enum
import math
import sympy

In [164]:
class Pivoting(Enum):
  NO_PIVOTING = 0,
  PARTIAL_PIVOTING = 1,
  FULL_PIVOTING = 2

  def generate_permutation_matrices(self, A:np.ndarray[np.ndarray]) -> Tuple[np.ndarray[np.ndarray], np.ndarray[np.ndarray]]:
    match self:
      case Pivoting.NO_PIVOTING:
        return (np.identity(len(A)), np.identity(len(A)))
      case Pivoting.PARTIAL_PIVOTING:
        return (Pivoting.__generate_partial_pivot_matrix(A), np.identity(len(A)))
      case Pivoting.FULL_PIVOTING:
        return Pivoting.__generate_full_pivot_matrix(A)


  def __generate_partial_pivot_matrix(A:np.ndarray[np.ndarray]) -> np.ndarray[np.ndarray]:
    n = len(A)
    pivot_matrix = np.identity(n)
    for i in range(n):
      # Find the highest value in the column
      PA = pivot_matrix @ A
      index_highest_value = np.abs(PA[i:, i]).argmax() + i
      pivot_matrix[[i, index_highest_value]] = pivot_matrix[[index_highest_value, i]]
    return pivot_matrix

  def __generate_full_pivot_matrix(A:np.ndarray[np.ndarray]) -> Tuple[np.ndarray[np.ndarray], np.ndarray[np.ndarray]]:
    n = len(A)
    row_permutation_matrix = np.identity(n)
    column_permutation_matrix = np.identity(n)
    for i in range(n):
      PA = row_permutation_matrix @ A @ column_permutation_matrix

      highest_value = max(PA[i:, i:].min(), PA[i:, i:].max(), key=abs)
      max_value = np.where(PA[i:, i:] == highest_value)

      # Use the first found occurence if there is more than one
      index_row_highest_value = max_value[0][0]
      index_column_highest_value = max_value[1][0]

      row_permutation_matrix[[i, index_row_highest_value + i]] = row_permutation_matrix[[index_row_highest_value + i, i]]
      column_permutation_matrix[:, [i, index_column_highest_value + i]] = column_permutation_matrix[:, [index_column_highest_value + i, i]]
      
    return (row_permutation_matrix, column_permutation_matrix)

# Gaussian Elimination

In [152]:
def gauss_method(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.PARTIAL_PIVOTING) -> np.ndarray:
  P_rows, P_columns = pivoting.generate_permutation_matrices(A)

  PA = P_rows @ A @ P_columns
  Pb = P_rows @ b

  for index_col in range(len(A)):
    pivot = PA[index_col][index_col]
    for index_row_below in range(index_col + 1, len(A)):
      ratio = PA[index_row_below][index_col] / pivot
      
      for idx in range(index_col, len(A)):
        PA[index_row_below, idx] -= PA[index_col, idx] * ratio

      Pb[index_row_below] -= Pb[index_col] * ratio
  
  result = np.linalg.solve(PA, Pb)
  return P_columns @ result


In [166]:
A = np.array([
  [3,2,4],
  [1,1,2],
  [4,3,-2],], 
dtype=np.float64)
b = np.array([1,2,3], np.float64)

# {-3, 5, 0}
gauss_method(A, b, Pivoting.PARTIAL_PIVOTING)


array([-3.,  5.,  0.])

# LU Factoring Method

In [154]:
def is_possible_to_apply_LU_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) != 0 and is_possible_to_apply_LU_method(A[1:, 1:])
  return False

# https://jonshiach.github.io/ODEs-book/pages/5.3_LUP_decomposition.html
def lu_factoring(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_LU_method(A) == True

  P_rows, P_columns = pivoting.generate_permutation_matrices(A)

  PA = P_rows @ A @ P_columns
  Pb = P_rows @ b

  upper_matrix = PA
  lower_matrix = np.identity(len(A), np.float64)
  
  for index_col in range(len(upper_matrix)):
    pivot = upper_matrix[index_col, index_col] 
    
    for index_row_below in range(index_col + 1, len(upper_matrix)):
      ratio = upper_matrix[index_row_below, index_col]/pivot
      lower_matrix[index_row_below, index_col] = ratio

      for idx in range(index_col, len(A)):
        upper_matrix[index_row_below, idx] -= upper_matrix[index_col, idx] * ratio

  y = np.linalg.solve(lower_matrix, Pb)
  result = np.linalg.solve(upper_matrix, y)
  return P_columns @ result

# Cholesky method

In [158]:
def is_possible_to_apply_Cholesky_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) > 0.0 and np.allclose(A, A.T) and is_possible_to_apply_Cholesky_method(A[1:, 1:])
  return False

def cholesky_method(A:np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_Cholesky_method(A) == True

  P_rows, P_columns = pivoting.generate_permutation_matrices(A)

  PA = P_rows @ A @ P_columns
  Pb = P_rows @ b

  G = np.zeros(A.shape)

  for i in range(len(A)):
    for j in range(i + 1):
      sum = 0
      for k in range(j):
        sum += G[i][k] * G[j][k]
      
      if i == j:
        G[i][j] = math.sqrt(PA[i][i] - sum)
      else:
        G[i][j] = (PA[i][j] - sum) / G[j][j]

  G_transpose = G.T
  y = np.linalg.solve(G, Pb)
  x = np.linalg.solve(G_transpose, y)
  return P_columns @ x

# Gauss Jacobi

In [10]:
def is_possible_to_apply_Jacobi_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    spectral_radius_less_than_one = all(map(lambda x: abs(x) < 1.0, np.linalg.eigvals(A)))
    
    rows_criteria = all(map(lambda x: A[x, :].sum() - A[x, x] < A[x, x], range(len(A))))
    columns_criteria = all(map(lambda x: A[:, x].sum() - A[x, x] < A[x, x], range(len(A))))

    diagonally_dominant = rows_criteria or columns_criteria

    return spectral_radius_less_than_one or diagonally_dominant
  return False

def jacobi_method(A:np.ndarray, b:np.ndarray, x:np.ndarray, iterations: int = 10) -> np.ndarray:
  assert is_possible_to_apply_Jacobi_method(A) == True

  for _ in range(iterations):
    # Diagonal matrix
    D = np.diag(np.diag(A))
    
    # construct a diagonal matrix where each position is 1/(diagonal element of A)
    D_inv = np.linalg.inv(D)

    x = np.dot(D_inv, b - np.dot(A - D, x))

  return x

# Gaus Seidel

In [11]:
def is_possible_to_apply_Seidel_method(A:np.ndarray) -> bool:
  return is_possible_to_apply_Jacobi_method(A) # and meet_sassenfeld_criterion(A)

def seidel_method(A:np.ndarray, b: np.ndarray, x: np.ndarray, iterations : int = 10) -> np.ndarray:
  assert is_possible_to_apply_Seidel_method(A) == True

  for _ in range(iterations):
    for i in range(len(A)):
      x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

  return x

# Newton's Method

In [55]:
def is_possible_to_apply_Newton_method(A:np.ndarray) -> bool:
  pass

def generate_jacobi_function(functions: np.ndarray[sympy.core.Expr], vars : np.ndarray[sympy.Symbol]) -> Callable[[np.ndarray], np.ndarray]:
  jacobi_matrix = [[sympy.diff(f, var) for var in vars] for f in functions]
  def jacobi_function(x:np.ndarray) -> np.ndarray:
    return np.array([[f.subs(list(zip(vars, x))) for f in row ] for row in jacobi_matrix], dtype=np.float64)
  return jacobi_function

def generate_F_function(functions: np.ndarray[sympy.core.Expr], vars : np.ndarray[sympy.Symbol]) -> Callable[[np.ndarray], np.ndarray]:
  def F_function(x:np.ndarray) -> np.ndarray:
    return np.array([f.subs(list(zip(vars, x))) for f in functions], dtype=np.float64)
  return F_function

def newton_method(F: Callable[[np.ndarray], np.ndarray], J: Callable[[np.ndarray], np.ndarray], x:np.ndarray, iterations: int = 10) -> np.ndarray:
  for _ in range(iterations):
    s = gauss_method(J(x), -F(x))
    x = x + s
  return x


In [25]:
x, y = sympy.symbols('x,y')

f1 = x**2 + sympy.sin(y) - 3
f2 = 2*x + sympy.sqrt(y)

funcs = np.array([f1, f2])
vars = np.array([x, y])

jacobi = [[sympy.diff(func, var) for var in vars] for func in funcs]

jacobi
#F = np.ndarray

[[2*x, cos(y)], [2, 1/(2*sqrt(y))]]

In [52]:
x, x_2 = sympy.symbols('x,x_2')

f1 = 4.*x - x**3. + x_2
f2 = (-x**2.)/9. + (4.*x_2 - x_2**2.)/4. + 1.

funcs = np.array([f1, f2])
vars = np.array([x, x_2])

initial_guess = np.array([-1.0, -2.0])

newton_method(generate_F_function(funcs, vars), generate_jacobi_function(funcs, vars), initial_guess)


array([ 1.93177027, -0.51822368])

In [56]:
x,y = sympy.symbols('x,y')

f1 = (x-2)**2 + (y+3)**2 - 4
f2 = y + 1

funcs = np.array([f1, f2])
vars = np.array([x, y])
F = generate_F_function(funcs, vars)
J = generate_jacobi_function(funcs, vars)

initial_guess = np.array([0.0, 0.0])

newton_method(F, J, initial_guess)

array([ 1.99755859, -1.        ])

# Gauss with pivoting

In [116]:
A = np.array([[1,-7], [5,2]], dtype=np.float64)
b = np.array([-11,-18], dtype=np.float64)
# Solution x=-4, y=1
gauss_method(A,b, Pivoting.PARTIAL_PIVOTING)


array([-4.,  1.])

In [137]:
A = np.array([
  [3,2,4],
  [1,1,2],
  [4,3,-2],], 
dtype=np.float64)
b = np.array([1,2,3], np.float64)

# {-3, 5, 0}
lu_factoring(A, b, Pivoting.FULL_PIVOTING)

array([-3.,  5.,  0.])

In [119]:
# 1x + 2y - 2z = -15
# 2x + 1y - 5z = -21
# 1x - 4y + 1z = +18 
# Solution {-1, -4, 3}

A = np.array([
  [1, 2, -2],
  [2, 1, -5],
  [1, -4, 1]
], dtype=np.float64)

b = np.array([-15, -21, 18], dtype=np.float64)

gauss_method(A, b, Pivoting.FULL_PIVOTING)

array([-1., -4.,  3.])

In [122]:
A = np.array([
  [1.,    2.,     1., -1.],
  [3./2., 1.,     2., 2.],
  [4.,    4.,     3., 4.],
  [2./5., 0.,  1./5., 1.]
], dtype=np.float64)

b = np.array([5., 8., 22., 3.], dtype=np.float64)

# x = 16
# y = -6
# z = -2
# t = -3

gauss_method(A, b, Pivoting.FULL_PIVOTING)

array([16., -6., -2., -3.])

In [163]:
A = np.array([
  [1.,  1.,  0.],
  [1.,  2., -1.],
  [0., -1.,  3.],
], dtype=np.float64)

b = np.array([2., 1., 5.], dtype=np.float64)

# x = 1
# y = 1
# z = 2

cholesky_method(A, b, Pivoting.FULL_PIVOTING)

array([1., 1., 2.])

In [142]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

# x = 1
# y = 0
# z = 1

# cholesky_method(A, b)
lu_factoring(A, b, Pivoting.FULL_PIVOTING)

array([1.0000000e+00, 8.8817842e-16, 1.0000000e+00])

In [20]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

jacobi_method(A, b, np.array([0., 0., 0.]), 20)

array([0.63911074, 0.18195205, 0.74674592])

In [21]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

seidel_method(A, b, np.array([0., 0., 0.]), 20)

array([0.82057774, 0.08823726, 0.88104021])